In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from flask import Flask, request, jsonify

In [ ]:
# Preprocessing Data
def preprocess_text(text):
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    nltk.download('punkt')
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    return ' '.join([word for word in tokens if word not in stop_words])



In [ ]:
# Example Dataset
data = {
    "greetings": ["hello", "hi", "hey", "good morning", "good evening"],
    "farewell": ["bye", "goodbye", "see you later", "take care"],
    "query": ["what is your name?", "who are you?", "what can you do?"]
}

labels = ["greetings", "farewell", "query"]



In [ ]:
# Training the Intent Classifier
vectorizer = TfidfVectorizer(preprocessor=preprocess_text)
X = vectorizer.fit_transform(data["greetings"] + data["farewell"] + data["query"])
y = ["greetings"] * len(data["greetings"]) + ["farewell"] * len(data["farewell"]) + ["query"] * len(data["query"])

classifier = LinearSVC()
classifier.fit(X, y)



In [ ]:
# Chatbot Functionality
def chatbot_response(query):
    query_vectorized = vectorizer.transform([query])
    intent = classifier.predict(query_vectorized)[0]
    responses = {
        "greetings": "Hello! How can I assist you today?",
        "farewell": "Goodbye! Have a great day!",
        "query": "I'm here to help with your questions. What do you need?"
    }
    return responses.get(intent, "I'm sorry, I didn't understand that.")


In [ ]:
# Flask API
app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    response = chatbot_response(user_input)
    return jsonify({"response": response})

if __name__ == '__main__':
    app.run(debug=True)
